# Distilling Knowledge in Multiple Students  using Generative models

In [1]:
# !nvidia-smi

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Teacher's Training

In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

print(teacher.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [4]:

# Train the teacher model as usual
epochs = 0
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


#checking teachers test accuracy
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
# Normalize the values
X_test /= 255


### Evaluation 

In [5]:
teacher.load_weights("Teacher_MNIST_98.h5")

In [6]:
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [7]:
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 3s 3ms/step - loss: 0.0541 - accuracy: 0.9869


[0.03862640634179115, 0.9898999929428101]

### Dense Vector split

In [8]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 2 Students case
# ---------------------------------------------
s1Train=train_dense[:,:128]
s2Train=train_dense[:,128:]

s1Val=val_dense[:,:128]
s2Val=val_dense[:,128:]

### Student

In [10]:
def define_student(name):

    student1 = Sequential()
    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
#     student1.add(Dense(32, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(128,name='req'+name))

    student1.compile('adam',loss='mean_squared_error',metrics=['accuracy'])
    
    return student1



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 16)        2320      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 16)        2320      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 4, 4, 16)         

In [11]:
s1=define_student("s1")
s2=define_student('s2')

In [12]:
s1.fit(X_train, s1Train,batch_size=128, epochs=10, verbose=1, validation_data=(X_val, s1Val))
s2.fit(X_train, s2Train,batch_size=128, epochs=10, verbose=1, validation_data=(X_val, s2Val))

Epoch 1/10
375/375 [==============================] - 4s 8ms/step - loss: 1.5284 - accuracy: 0.0958 - val_loss: 0.4916 - val_accuracy: 0.3662
Epoch 2/10
375/375 [==============================] - 2s 5ms/step - loss: 0.5698 - accuracy: 0.3437 - val_loss: 0.3339 - val_accuracy: 0.4320
Epoch 3/10
375/375 [==============================] - 2s 5ms/step - loss: 0.4500 - accuracy: 0.3989 - val_loss: 0.2837 - val_accuracy: 0.4583
Epoch 4/10
375/375 [==============================] - 2s 6ms/step - loss: 0.4158 - accuracy: 0.4037 - val_loss: 0.2620 - val_accuracy: 0.4708
Epoch 5/10
375/375 [==============================] - 3s 9ms/step - loss: 0.3941 - accuracy: 0.4095 - val_loss: 0.2573 - val_accuracy: 0.4787
Epoch 6/10
375/375 [==============================] - 3s 7ms/step - loss: 0.3820 - accuracy: 0.4159 - val_loss: 0.2388 - val_accuracy: 0.4802
Epoch 7/10
375/375 [==============================] - 3s 9ms/step - loss: 0.3684 - accuracy: 0.4165 - val_loss: 0.2475 - val_accuracy: 0.4918
Epoch 

## **2 Students**

In [13]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
output=tensorflow.keras.layers.concatenate([o1,o2])

output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)

mm2=Model([s1.get_layer("s1").input,s2.get_layer("s2").input], output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm2.get_layer('d1').set_weights(my_weights)

In [14]:
i=0
for l in mm2.layers[:len(mm2.layers)-1]:
    l.trainable=False

mm2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [16]:
batch_size = 128
mm2_history=mm2.fit([X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val,X_val], Y_val))

Epoch 1/10
375/375 [==============================] - 3s 8ms/step - loss: 0.0984 - accuracy: 0.9696 - val_loss: 0.0558 - val_accuracy: 0.9819
Epoch 2/10
375/375 [==============================] - 3s 8ms/step - loss: 0.1003 - accuracy: 0.9697 - val_loss: 0.0557 - val_accuracy: 0.9815
Epoch 3/10
375/375 [==============================] - 3s 8ms/step - loss: 0.0987 - accuracy: 0.9696 - val_loss: 0.0550 - val_accuracy: 0.9821
Epoch 4/10
375/375 [==============================] - 3s 8ms/step - loss: 0.1005 - accuracy: 0.9693 - val_loss: 0.0554 - val_accuracy: 0.9816
Epoch 5/10
375/375 [==============================] - 3s 7ms/step - loss: 0.0982 - accuracy: 0.9694 - val_loss: 0.0562 - val_accuracy: 0.9818
Epoch 6/10
375/375 [==============================] - 3s 9ms/step - loss: 0.0977 - accuracy: 0.9696 - val_loss: 0.0549 - val_accuracy: 0.9812
Epoch 7/10
375/375 [==============================] - 3s 7ms/step - loss: 0.0987 - accuracy: 0.9701 - val_loss: 0.0569 - val_accuracy: 0.9808
Epoch 

In [17]:
loss, acc = mm2.evaluate([X_test,X_test], Y_test, verbose=1)
loss, acc 

313/313 [==============================] - 1s 4ms/step - loss: 0.0462 - accuracy: 0.9848


(0.046155378222465515, 0.9847999811172485)

In [18]:
mm2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s1 (Conv2D)                     (None, 26, 26, 16)   160         s1_input[0][0]                   
__________________________________________________________________________________________________
s2 (Conv2D)                     (None, 26, 26, 16)   160         s2_input[0][0]                   
____________________________________________________________________________________________